# Chordia Mode - With lags of independent variables

Same model but with lags of independent variable

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.stats.stattools import durbin_watson

In [2]:
pd.set_option('use_inf_as_na', True)

In [3]:
df=pd.read_excel("BBREIT-1.07.xlsx",sheet_name="PROPORTIONAL_BID_ASK_SPREAD", index_col="Dates")
df.dropna(axis=1,inplace=True)

In [4]:
for i in df:
    df[i]= np.log(df[i])-np.log(df[i]).shift(1)

C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\ravn_\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
df.drop(index=df.index[0], axis=0, inplace=True)

In [6]:
df.isnull().values.sum()

55

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().values.sum()

0

In [9]:
q =df.quantile(0.9999)
b =df.quantile(0.0001)

In [10]:
df=df[(df < q) & (df > b)]

In [11]:
df["Sum"]=df.sum(axis=1)

In [12]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    df["1lag"]=df[i].shift(1) 
    df["VOL"]= ((df[i]-df[i].shift(1))/df[i].shift(1))**2 
    df["M"]=((df["Sum"]-df[i])/157) 
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","VOL","Mlag","Mlead","1lag"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","VOL","Mlag","Mlead","1lag"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     41.19
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.33e-35
Time:                        12:11:43   Log-Likelihood:                -876.92
No. Observations:                 516   AIC:                             1766.
Df Residuals:                     510   BIC:                             1791.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0162      0.059     -0.275      0.7

(6.499449708352748, 0.26060548070151074, 1.3011641888757477, 0.2619588918765755)
2.2552400290627053
                            OLS Regression Results                            
Dep. Variable:          EQR US Equity   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     28.57
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.44e-25
Time:                        12:11:43   Log-Likelihood:                -536.60
No. Observations:                 516   AIC:                             1085.
Df Residuals:                     510   BIC:                             1111.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(1.7621036548777576, 0.8809847327205378, 0.34951638935008444, 0.8825775036835709)
2.234504843591832
                            OLS Regression Results                            
Dep. Variable:          UDR US Equity   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     29.92
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.11e-26
Time:                        12:11:43   Log-Likelihood:                -377.75
No. Observations:                 516   AIC:                             767.5
Df Residuals:                     510   BIC:                             793.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

(7.261295266283219, 0.2019213989964757, 1.4558595803096144, 0.2027674608859518)
2.292316185952712
                            OLS Regression Results                            
Dep. Variable:          LSI US Equity   R-squared:                       0.213
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     27.55
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.02e-24
Time:                        12:11:44   Log-Likelihood:                -823.80
No. Observations:                 516   AIC:                             1660.
Df Residuals:                     510   BIC:                             1685.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

(11.723514407853699, 0.038779088638482576, 2.3713151490633417, 0.03830019688845853)
2.1942906544758505
                            OLS Regression Results                            
Dep. Variable:          EGP US Equity   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     33.37
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.73e-29
Time:                        12:11:44   Log-Likelihood:                -913.67
No. Observations:                 516   AIC:                             1839.
Df Residuals:                     510   BIC:                             1865.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

...

(9.50623893688552, 0.09049743846231949, 1.914409309854251, 0.09033267013071156)
2.1812053472996715
                            OLS Regression Results                            
Dep. Variable:         CHCT US Equity   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     25.53
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           5.17e-23
Time:                        12:11:45   Log-Likelihood:                -805.24
No. Observations:                 516   AIC:                             1622.
Df Residuals:                     510   BIC:                             1648.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

(17.063193917288405, 0.004381494253816121, 3.488309057069031, 0.004135397051451832)
2.2060118145735963
                            OLS Regression Results                            
Dep. Variable:         CLDT US Equity   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     19.60
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           7.07e-18
Time:                        12:11:45   Log-Likelihood:                -459.38
No. Observations:                 516   AIC:                             930.8
Df Residuals:                     510   BIC:                             956.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

(10.535016287881565, 0.061419732527561104, 2.1259072260007046, 0.06104291466042357)
2.0737763753843663
                            OLS Regression Results                            
Dep. Variable:         PINE US Equity   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     31.45
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           6.18e-28
Time:                        12:11:45   Log-Likelihood:                -706.45
No. Observations:                 516   AIC:                             1425.
Df Residuals:                     510   BIC:                             1450.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

                            OLS Regression Results                            
Dep. Variable:                    Sum   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     53.40
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           8.53e-22
Time:                        12:11:46   Log-Likelihood:                -2341.7
No. Observations:                 516   AIC:                             4689.
Df Residuals:                     513   BIC:                             4702.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2677      1.001      1.267      0.2

C:\Users\ravn_\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


### Summary Results - Model with one lag of dependent variable

Coefficient remains similar. R2 very much enlarged. Because we add lagged variable of dependent variable?
Durbin Watson reduced to 2.2 from 2.8

In [13]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.8103

In [14]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.2448
dtype: float64

In [15]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.2066
dtype: float64

In [16]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.0871

In [17]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.2853

# Model with two lags

In [19]:
Coefficients = []
Pvalues = []
Rsquared_adj = []
Bpagan = []
DBWatson = []
STD = []

for i in df:
    df["1lag"]=df[i].shift(1) 
    df["2lag"]=df[i].shift(2)
    df["VOL"]= ((df[i]-df[i].shift(1))/df[i].shift(1))**2 
    df["M"]=((df["Sum"]-df[i])/157) 
    df["Mlag"]=df["M"].shift(1)
    df["Mlead"]=df["M"].shift(-1)
    df.replace([np.inf], np.nan, inplace=True)
    df=df.fillna(0)
        
        
      
    y = df[i]
    x = df[["M","VOL","Mlag","Mlead","1lag","2lag"]]
    x = sm.add_constant(x)
    model = sm.OLS(y,x, missing = 'drop') 
    results = model.fit()
    print(results.summary())
    print(sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x))
    print(durbin_watson(results.resid))
    
    
    # Filling the lists with the content specified
    coefficients=results.params 
    Coefficients.append(coefficients)
    
    pvalues=results.pvalues
    Pvalues.append(pvalues)
    
    rsquared_adj = results.rsquared_adj
    Rsquared_adj.append(rsquared_adj)
    
    bpagan = sm.stats.diagnostic.het_breuschpagan(results.resid, exog_het = x)
    Bpagan.append(bpagan)
    
    std=results.bse
    STD.append(std)
    
    dbwatson = durbin_watson(results.resid)
    DBWatson.append(dbwatson)
    
    # CLearing the dynamic variables after use. Mostly for practical reuse-reasons
    Drop=df[["M","VOL","Mlag","Mlead","1lag","2lag"]]
    df.drop(columns=Drop, inplace=True)

                            OLS Regression Results                            
Dep. Variable:          AMT US Equity   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     42.85
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.41e-42
Time:                        12:14:17   Log-Likelihood:                -858.94
No. Observations:                 516   AIC:                             1732.
Df Residuals:                     509   BIC:                             1762.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0123      0.057     -0.216      0.8

(7.720431089986784, 0.25930903198525546, 1.2885623271035556, 0.26065465767227114)
2.1144909839386523
                            OLS Regression Results                            
Dep. Variable:          EQR US Equity   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     30.34
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           3.73e-31
Time:                        12:14:17   Log-Likelihood:                -521.43
No. Observations:                 516   AIC:                             1057.
Df Residuals:                     509   BIC:                             1087.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

(10.246423214460293, 0.11465087169244649, 1.7186991371398834, 0.11454555745852421)
2.070300541491032
                            OLS Regression Results                            
Dep. Variable:          ESS US Equity   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     31.97
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.13e-32
Time:                        12:14:17   Log-Likelihood:                -872.97
No. Observations:                 516   AIC:                             1760.
Df Residuals:                     509   BIC:                             1790.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

                            OLS Regression Results                            
Dep. Variable:          BXP US Equity   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     49.18
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.29e-47
Time:                        12:14:17   Log-Likelihood:                -728.21
No. Observations:                 516   AIC:                             1470.
Df Residuals:                     509   BIC:                             1500.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0143      0.044     -0.322      0.7

                            OLS Regression Results                            
Dep. Variable:         REXR US Equity   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     49.12
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           1.44e-47
Time:                        12:14:17   Log-Likelihood:                -581.07
No. Observations:                 516   AIC:                             1176.
Df Residuals:                     509   BIC:                             1206.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0270      0.033     -0.811      0.4

...

(22.085693747194828, 0.0011683288637015466, 3.7933766967962805, 0.001044766453671512)
2.0900504580668544
                            OLS Regression Results                            
Dep. Variable:          RPT US Equity   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     58.23
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           9.47e-55
Time:                        12:14:20   Log-Likelihood:                -311.07
No. Observations:                 516   AIC:                             636.1
Df Residuals:                     509   BIC:                             665.9
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

(26.15685775106685, 0.00020814910801764222, 4.529967332728684, 0.0001733523356635126)
2.0724684737910573
                            OLS Regression Results                            
Dep. Variable:         NXDT US Equity   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.248
Method:                 Least Squares   F-statistic:                     29.27
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           3.81e-30
Time:                        12:14:20   Log-Likelihood:                -593.47
No. Observations:                 516   AIC:                             1201.
Df Residuals:                     509   BIC:                             1231.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

(21.538074460583477, 0.0014677180571184972, 3.695222130762782, 0.001323368436173591)
2.1329909145927215
                            OLS Regression Results                            
Dep. Variable:          FSP US Equity   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     31.53
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.86e-32
Time:                        12:14:20   Log-Likelihood:                -107.92
No. Observations:                 516   AIC:                             229.8
Df Residuals:                     509   BIC:                             259.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

(-84.08712457521058, 1.0, nan, array(nan))
2.6093537263377398
                            OLS Regression Results                            
Dep. Variable:         AAIC US Equity   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     46.94
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           8.89e-46
Time:                        12:14:20   Log-Likelihood:                -246.72
No. Observations:                 516   AIC:                             507.4
Df Residuals:                     509   BIC:                             537.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

(11.386936990989057, 0.0771289522224933, 1.914321868011148, 0.07669866453562253)
2.024809082285724
                            OLS Regression Results                            
Dep. Variable:                    Sum   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     45.90
Date:                Sat, 22 Oct 2022   Prob (F-statistic):           2.77e-26
Time:                        12:14:20   Log-Likelihood:                -2329.0
No. Observations:                 516   AIC:                             4666.
Df Residuals:                     512   BIC:                             4683.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

### Summary Results - Model with two lags of dependent variable


In [20]:
Coef=pd.DataFrame(Coefficients)
round(Coef["M"].mean(),4)

0.7559

In [21]:
R2=pd.DataFrame(Rsquared_adj)
round(R2.mean(),4)

0    0.2915
dtype: float64

In [22]:
DBW=pd.DataFrame(DBWatson)
round(DBW.mean(),4)

0    2.0797
dtype: float64

In [23]:
BP=pd.DataFrame(Bpagan)
round(BP[1].mean(),4)

0.2369

In [24]:
PV=pd.DataFrame(Pvalues)
round(PV["M"].mean(),4)

0.0984